In [ ]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline


In [ ]:
from Coil_contribution import Coil_contribution


coils = Coil_contribution("coils_symmetric.txt")

#upper plasma coil (2x symmetric)
rp=4.0
zp=0.25
jp=-2.0e4

# B0 coils (two separately)
rp01=3.8
zp01=0.0
jp01=-2.0e4
rp02=3.8
zp02=-0.0 
jp02=-2.0e4

def eval_coil_psi(R,Z):
    '''Evaluates coil field '''
    psi_c = coils.eval_coil_field(R,Z)
    
    return psi_c


def eval_full_psi(R,Z):
    '''Evaluates coil field plus additional field from "coils" modelling for the plasma current '''
    psi_c = eval_coil_psi(R,Z)
    

    psi_p = coils.eval_multi_coil_field(R,Z,r_coils=[rp,rp],z_coils=[zp,-zp],j_coils=[jp,jp])

    
    psi=psi_c+psi_p
    
    return psi


def eval_full_Bpol(R,Z):
    '''Evaluates coil field plus additional field from "coils" modelling for the plasma current '''
    dpsi_dr_c,dpsi_dz_c = coils.eval_grad_coil_field(R,Z)
    
    dpsi_dr_p,dpsi_dz_p = coils.eval_grad_multi_coil_field(R,Z,r_coils=[rp,rp],z_coils=[zp,-zp],j_coils=[jp,jp])

    dpsi_dr=dpsi_dr_c+dpsi_dr_p
    dpsi_dz=dpsi_dz_c+dpsi_dz_p
    
    #Bpol_r=-dpsi_dz/R,Bpol_z=dpsi_dr/R
    return -dpsi_dz/R,dpsi_dr/R

def eval_psi_B0(R,Z):
    '''
    Evaluates coil field plus additional field from "coils" modelling for the plasma current. 
    Idea: this psi can be different to eval_full_psi, but should have the same  plasma current.
    '''
    psi_c = eval_coil_psi(R,Z)
    
    #use B0 coils

    psi_p1 = coils.eval_single_coil_field(R,Z,rp01,zp01,jp01)


    psi_p2 = coils.eval_single_coil_field(R,Z,rp02,zp02,jp02)
    
    psi=psi_c+psi_p1+psi_p2
    
    return psi

def eval_B0pol(R,Z):
    '''Evaluates coil field plus additional field from "coils" modelling for the plasma current '''
    dpsi_dr_c,dpsi_dz_c = coils.eval_grad_coil_field(R,Z)
    
    #use B0 coils
    
    dpsi_dr_p1,dpsi_dz_p1 = coils.eval_grad_single_coil_field(R,Z,rp01,zp01,jp01)

    dpsi_dr_p2,dpsi_dz_p2 = coils.eval_grad_single_coil_field(R,Z,rp02,zp02,jp02)
    
    dpsi_dr=dpsi_dr_c+dpsi_dr_p1+dpsi_dr_p2
    dpsi_dz=dpsi_dz_c+dpsi_dz_p1+dpsi_dz_p2
    
    #Bpol_r=-dpsi_dz/R,Bpol_z=dpsi_dr/R
    return -dpsi_dz/R,dpsi_dr/R


def eval_Bcoil(R,Z):
    '''Evaluates coil field plus additional field from "coils" modelling for the plasma current '''
    dpsi_dr,dpsi_dz = coils.eval_grad_coil_field(R,Z)
    
    #Bpol_r=-dpsi_dz/R,Bpol_z=dpsi_dr/R
    return -dpsi_dz/R,dpsi_dr/R


In [ ]:

def get_RZ_grid( Rlim=[2,5], Zlim=[-2,2], h_RZ=1e-2):
    '''Get a meshgrid of a given domain size and uniform approximate grid size h'''
    Nr=int((Rlim[1]-Rlim[0])/h_RZ)
    Nz=int((Zlim[1]-Zlim[0])/h_RZ)
    r = np.linspace(Rlim[0], Rlim[1], Nr)
    z = np.linspace(Zlim[0], Zlim[1], Nz)
    R, Z = np.meshgrid(r, z)
    return R,Z


In [ ]:
# visualize coil field

R,Z = get_RZ_grid(Rlim=[0.1,11],Zlim=[-7,7],h_RZ=0.1)
psi_c= eval_coil_psi(R,Z)
    

    
    
fig,ax= plt.subplots(figsize=(10,10))
    
im=ax.contourf(R, Z, psi_c, 120, cmap='jet',alpha=0.4)
ax.contour(R, Z, psi_c, 120,cmap='jet')

ax.plot(coils.coil_arr[:,0],coils.coil_arr[:,1],'ks')
plt.xlabel('R')
plt.ylabel('Z')
plt.title('Plot of the poloidal coil field  (psi) with all external coils')
fig.colorbar(im)
ax.axis("equal")

In [ ]:
#visualize coild field in default domain
R,Z = get_RZ_grid()
psi_c= eval_coil_psi(R,Z)
    
fig,ax= plt.subplots(figsize=(10,10))
    
im=ax.contourf(R, Z, psi_c, 40, cmap='jet',alpha=0.4)
ax.contour(R, Z, psi_c, 40,cmap='jet')

#ax.plot(coils.coil_arr[:,0],coils.coil_arr[:,1],'bs')
plt.xlabel('R')
plt.ylabel('Z')
plt.title('Plot of the poloidal coil field (psi) in a smaller domain')
fig.colorbar(im)
ax.axis("equal")

In [ ]:
#visualize full field 
R,Z = get_RZ_grid()
psi= eval_full_psi(R,Z)
    
fig,ax= plt.subplots(figsize=(10,10))
    
im=ax.contourf(R, Z, psi, 40, cmap='jet',alpha=0.4)
ax.contour(R, Z, psi, 40,cmap='jet')


plt.xlabel('R')
plt.ylabel('Z')
plt.title('Plot of the reference poloidal field (psi), coil field plus "plasma current" ')
fig.colorbar(im)
ax.axis("equal")

In [ ]:

def eval_1d_fourier(t,coef_c=[],coef_s=[]):
    '''Evaluates cosine/sine series, first coef_c is mode m=0, of coef_s is mode m=1'''
    x=0.0*t
    for m in range(0,len(coef_c)):
        x+=coef_c[m]*np.cos(m*t*2*np.pi)
    for m in range(0,len(coef_s)):
        x+=coef_s[m]*np.sin((m+1)*t*2*np.pi)
    return x


def eval_1d_fourier_dt(t,coef_c=[],coef_s=[]):
    '''Evaluates first derivative of cosine/sine series, first coef_c is mode m=0, of coef_s is mode m=1'''
    dxdt=0.0*t
    for m in range(0,len(coef_c)):
        dxdt+=-m*2*np.pi*coef_c[m]*np.sin(m*t*2*np.pi)
    for m in range(0,len(coef_s)):
        dxdt+=(m+1)*2*np.pi*coef_s[m]*np.cos((m+1)*t*2*np.pi)
    return dxdt


def eval_curve(x,xmap,N):
    '''
    Evaluates r,z of the curve defined by fourier coefficients 
    rcoef_c/s and zcoef_c/s , at N points. coefficients are mapped from 1d array x, using xmap dict.
    '''
    thet=np.linspace(0.,1.,N,endpoint=False)
    rr=eval_1d_fourier(thet,
                       coef_c=x[xmap["str_r_c"]:xmap["str_r_c"]+xmap["nr_c"]],
                       coef_s=x[xmap["str_r_s"]:xmap["str_r_s"]+xmap["nr_s"]])

    zz=eval_1d_fourier(thet,
                       coef_c=x[xmap["str_z_c"]:xmap["str_z_c"]+xmap["nz_c"]],
                       coef_s=x[xmap["str_z_s"]:xmap["str_z_s"]+xmap["nz_s"]])
    return rr,zz


def eval_curve_dt(x,xmap,N):
    '''
    Evaluates r,z of the curve defined by fourier coefficients 
    rcoef_c/s and zcoef_c/s , at N points. coefficients are mapped from 1d array x, using xmap dict.
    '''
    thet=np.linspace(0.,1.,N,endpoint=False)
    dr_dt=eval_1d_fourier_dt(thet,
                       coef_c=x[xmap["str_r_c"]:xmap["str_r_c"]+xmap["nr_c"]],
                       coef_s=x[xmap["str_r_s"]:xmap["str_r_s"]+xmap["nr_s"]])

    dz_dt=eval_1d_fourier_dt(thet,
                       coef_c=x[xmap["str_z_c"]:xmap["str_z_c"]+xmap["nz_c"]],
                       coef_s=x[xmap["str_z_s"]:xmap["str_z_s"]+xmap["nz_s"]])
    return dr_dt,dz_dt



In [ ]:
# FIND A PARAMETRIZATION OF THE CLOSED FLUX SURFACE

def minf(x_in,xmap,psi_goal,N):
    '''
    function for the minimizer: evaluate psi on the curve and 
    return the normalized square sum of the difference to psi_goal
    '''
    rr,zz=eval_curve(x_in,xmap,N)
    psi_curve=eval_full_psi(rr,zz)
    return 0.5*np.sum((psi_curve-psi_goal)**2)/(psi_goal**2*N)

def minf_point(x,psi_goal):
    return 0.5*(eval_full_psi(x[0],x[1])-psi_goal)**2/(psi_goal**2)

def find_flux_surface(**kwargs):
    '''
    Main function to find a closed flux surface (with parametrization of the angle!) of the coil+plasma field. 
    Visualization helps if default parameters would change.
    Defines a contour(=flux surface) of psi at a given point (r_fs,z_fs), uses the contour object and extract the closed contour.
    The contour object gives a bounding box which is used to initialize the fourier series of the curve.
    Then the coefficients are optimized to match the contour level, 
    by minimizing the difference of the flux evaluated at the curve to the flux of the contour.
    '''
    from scipy.optimize import minimize

    
    R,Z = get_RZ_grid(**kwargs)
    psi= eval_full_psi(R,Z)
    
    #select flux surface going through the point:
    r_fs=3.15
    z_fs=0.
    psi_fs =eval_full_psi(r_fs,z_fs) 
    
    
    fig,ax= plt.subplots(figsize=(10,10))
    
    im=ax.contourf(R, Z, psi, 40, cmap='jet',alpha=0.4)
    ax.contour(R, Z, psi, 40,cmap='jet')
   
    
    ax.contour(R,Z, psi, [psi_fs], colors='k', linewidths=2., linestyles='solid')
    

    # get the contour object
    fs = ax.contour(R, Z, psi, [psi_fs])
    
    fs_list = fs.collections[0].get_paths()
    # Sort the paths by its length. Assume main one is the longest(?)
    fs_list.sort(key=len, reverse=True)
    
    fs_coord = fs_list[0].vertices
    fs_r =fs_coord[:, 0]
    fs_z = fs_coord[:, 1]
    
    
    idx_left=np.argmin(fs_r[:])
    idx_right=np.argmax(fs_r[:])
    idx_lower=np.argmin(fs_z[:])
    idx_upper=np.argmax(fs_z[:])
    
    #print ("bound. box, left (%f,%f),right (%f,%f),lower (%f,%f),upper (%f,%f)" % (fs_r[idx_left],fs_z[idx_left],fs_r[idx_right],fs_z[idx_right],fs_r[idx_lower],fs_z[idx_lower],fs_r[idx_upper],fs_z[idx_upper]))
    #ax.plot(fs_r[0:-1:10],fs_z[0:-1:10],'bx')

    
    psi_back=eval_full_psi(fs_r,fs_z)
    
    print ("psi_fs %e, max abs (psi_contour-psi_fs)= %e" % (psi_fs,np.amax(np.abs(psi_back-psi_fs))))
    
    #testing minimizer with one point:
    x0_point=[3.5,1.2]
    diff_psi=minf_point(x0_point,psi_fs)
    print ("psi_fs %e, initial residual,sqrt||(psi_point-psi_fs)^2||= %e " % (psi_fs,np.sqrt(diff_psi)))
    
    res_point=minimize(minf_point, x0_point, args=(psi_fs),tol=1.0e-14)
    
    print("message minimizer: " +res_point.message)
    diff_psi=minf_point(res_point.x,psi_fs)
    print ("psi_fs %e, final residual,sqrt||(psi_point-psi_fs)^2||= %e " % (psi_fs,np.sqrt(diff_psi)))
    
    # plt.plot([x0_point[0],res_point.x[0]],[x0_point[1],res_point.x[1]],'ro')
    
    # initialize Dshaped up-down symmetric curve inside bounding box of contour 
    m_max=9
    rcoef_c0=np.zeros(m_max+1)
    rcoef_s0=[] #up-down symmetric
    zcoef_c0=[] #up-down symmetric
    #rcoef_s0=np.zeros(m_max)   #unsymmetric
    #zcoef_c0=np.zeros(m_max+1) #unsymmetric
    zcoef_s0=np.zeros(m_max)
    
    rcoef_c0[2]=0.1 #m=2
    rcoef_c0[1]=0.5*(fs_r[idx_right]-fs_r[idx_left]) # m=1
    rcoef_c0[0]=0.5*(fs_r[idx_right]+fs_r[idx_left]) - rcoef_c0[2] # m=0
    zcoef_s0[0]=fs_z[idx_upper] #m=1
    zcoef_s0[1]=-0.1 #m=2
    
    
    # BUILD 1D solution vector x0
    x0=np.concatenate((rcoef_c0,rcoef_s0,zcoef_c0,zcoef_s0))
    xmap={}
    xmap["nr_c"]=len(rcoef_c0)
    xmap["nr_s"]=len(rcoef_s0)
    xmap["nz_c"]=len(zcoef_c0)
    xmap["nz_s"]=len(zcoef_s0)
    xmap["str_r_c"]=0
    xmap["str_r_s"]=0+xmap["nr_c"]
    xmap["str_z_c"]=xmap["str_r_s"]+xmap["nr_s"]
    xmap["str_z_s"]=xmap["str_z_c"]+xmap["nz_c"]
    
    # number of points for evaluating the "distance" in psi on the curve
    N=41
    N_post=251
    
    rr,zz=eval_curve(x0,xmap,N)
    
    #plt.plot(rr,zz,'b.')
    
    diff_psi=minf(x0,xmap,psi_fs,N_post)
    print ("psi_fs %e, initial residual,||(psi_curve-psi_fs)^2||= %e " % (psi_fs,diff_psi))
    
    res=minimize(minf, x0, args=(xmap,psi_fs,N),tol=1.0e-14)
      
    print("message minimizer: " + res.message)
    #res=minimize(minf, x0, args=(nr,nz,psi_fs,N),method='Nelder-Mead',options={'xatol': 1e-08, 'fatol': 1e-08})
    #res=least_squares(minf_LS, x0, args=(nr,nz,psi_fs,N),xtol=1.0e-12)
    
    #post-processing

    diff_psi=minf(res.x,xmap,psi_fs,N_post)
    print ("psi_fs %e, final residual ,||(psi_curve-psi_fs)^2||= %e " % (psi_fs,diff_psi))
    
    rr,zz=eval_curve(res.x,xmap,N_post)

    psi_back=eval_full_psi(rr,zz)
    print ("psi_fs %e, max abs (psi_fit-psi_fs)= %e" % (psi_fs,np.amax(np.abs(psi_back-psi_fs))))
    
    
    # visualize result
    rr,zz=eval_curve(res.x,xmap,50)
    plt.plot(rr,zz,'bo')
    
    # visualize normalized Bpol
    # Bpol_r,Bpol_z = eval_full_Bpol(rr,zz)
    # absBpol=np.sqrt(Bpol_r**2+Bpol_z**2)
    # ax.quiver(rr,zz,Bpol_r,Bpol_z)
    
    plt.xlabel('R')
    plt.ylabel('Z')
    #plt.title('Plot of the poloidal field with "plasma" ')
    plt.title(('Plot of the reference poloidal field (coils+ "plasma") \n with fitted closed flux surface through point (%4.2f,%4.2f)'%(r_fs,z_fs)))
    fig.colorbar(im)
    ax.axis("equal")
    plt.show()
    
    return res.x,xmap,psi_fs
    

x_final,xmap,psi_fs=find_flux_surface()

In [ ]:
#some more post-processing
N_post=251
    
    
rr,zz=eval_curve(x_final,xmap,N_post)
dr_dt,dz_dt=eval_curve_dt(x_final,xmap,N_post)
    
    
n_r=dz_dt/np.sqrt(dr_dt**2+dz_dt**2)
n_z=-dr_dt/np.sqrt(dr_dt**2+dz_dt**2)
Bpol_r,Bpol_z = eval_full_Bpol(rr,zz)
print ("maximum |Bpol|= %e" % (np.amax(np.sqrt(Bpol_r**2+Bpol_z**2))))
print ("maximum error in B.n/|B|= %e" % (np.amax(np.abs(n_r*Bpol_r+n_z*Bpol_z)/np.sqrt(Bpol_r**2+Bpol_z**2))))
    
#circulation
print ("circulation of B,  J=int(B.t)/mu_0  %e" % (np.sum(dr_dt*Bpol_r+dz_dt*Bpol_z)*(1/N_post)/(4.0e-7*np.pi)))
    
B0_r,B0_z=eval_B0pol(rr,zz)
print ("circulation of B0 J=int(B0.t)/mu_0 %e" % (np.sum(dr_dt*B0_r+dz_dt*B0_z)*(1/N_post)/(4.0e-7*np.pi)))


In [ ]:


def plot_psi_B0(**kwargs):
 
    R,Z = get_RZ_grid(**kwargs)
    
    psi= eval_psi_B0(R,Z)
    
    fig,ax= plt.subplots(figsize=(10,10))
    
    im=ax.contourf(R, Z, psi, 40, cmap='jet',alpha=0.4)
    ax.contour(R, Z, psi, 40,cmap='jet')
    #ax.plot(coils.coil_arr[:,0],coils.coil_arr[:,1],'x')
    N=64
    rr,zz=eval_curve(x_final,xmap,N)
    
    psi_B0=eval_psi_B0(rr,zz)
    print ("psi_fs %e, max abs (psi_B0-psi_fs)= %e" % (psi_fs,np.amax(np.abs(psi_B0-psi_fs))))
    plt.plot(rr,zz,'b.')
    
    # visualize normalized Bpol 
    B0pol_r,B0pol_z = eval_B0pol(rr,zz)
    # absBpol=np.sqrt(Bpol_r**2+Bpol_z**2)
    ax.quiver(rr,zz,B0pol_r,B0pol_z,width=0.003)
    
    plt.xlabel('R')
    plt.ylabel('Z')
    plt.title('Plot of the poloidal B0 field (psi,arrows), \n coils plus same "plasma current" but placed differently. ')
    fig.colorbar(im)
    ax.axis("equal")
    plt.show()
    
plot_psi_B0()
N=128
thet=np.linspace(0.,1.,N,endpoint=False)
rr,zz=eval_curve(x_final,xmap,N)
psi_B0=eval_psi_B0(rr,zz)
plt.plot(thet,psi_B0,label="psi(B0)")
plt.plot(thet,0*thet+psi_fs,label="psi flux surface")
plt.xlabel('theta')
plt.ylabel('psi')
plt.title('Plot psi over the flux surface')
plt.legend(loc='upper right')
plt.show()

In [ ]:
# save data to file:
N=64
outdat=np.zeros((N,5))

rr,zz=eval_curve(x_final,xmap,N)
B0_R,B0_Z=eval_B0pol(rr,zz)
psi_B0=eval_psi_B0(rr,zz)
outdat[:,0]=rr     # R-coordinate
outdat[:,1]=zz     # Z-coordinate
outdat[:,2]=B0_R # Poloidal field component of B0 in R
outdat[:,3]=B0_Z # Poloidal field component of B0 in Z
outdat[:,4]=1./rr # torodial field component Bphi
#outdat[4,:]=psi_B0 # additional info 

np.savetxt(("outdat_%04d.csv"%(N)), outdat, delimiter=",")

    